## Chocolate Bar Analysis

In [1]:
# Import Libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

# Data Exploration

In [2]:
# Import data and dispaly first rows
df = pd.read_csv('C:/Users/nickm/Desktop/Coding/Projects/chocolate-bar-analysis/Chocolate bar ratings 2022.csv')
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/nickm/Desktop/Coding/Projects/python-pandas-project/Chocolate bar ratings 2022.csv'

In [ ]:
# Display DataFrame summary
df.info()

In [ ]:
# Print descriptive statistics of numerical columns
df.describe()

In [ ]:
# Count unique values in each column
df.nunique()

In [ ]:
# Display number of NAs for each column
df.isna().sum()

In [ ]:
# Show rows with NAs in 'Ingredients'
df[df['Ingredients'].isna()].head()

In [ ]:
# drop NA
df = df.dropna()

In [ ]:
# show 5 most frequent values for each column
for col in df.columns:
    print(df[col].value_counts().head())

# Recodes

### Rating - Categories

In [ ]:
# From Author: "The ratings are between 1 and 5 with 1 considered the lowest rating and 5 as the highest rating possible."
    # 4.0 - 5.0 = Outstanding
    # 3.5 - 3.9 = Highly Recommended
    # 3.0 - 3.49 = Recommended
    # 2.0 - 2.9 = Disappointing
    # 1.0 - 1.9 = Unpleasant
print(df['Rating'].unique())

def rating_fun(rating):
    if rating >= 4:
        return 'Outstanding'
    elif rating >= 3.5:
        return 'Highly Recommended'
    elif rating >= 3:
        return 'Recommended'
    elif rating >= 2.0:
        return 'Disappointing'
    else:
        return 'Unpleasant'
    
df['Rating Categories'] = df['Rating'].apply(rating_fun)

print(df['Rating Categories'].value_counts())
print(pd.crosstab(df['Rating'], df['Rating Categories']))

### Cacoa Content - Categories

In [ ]:
print(df['Cocoa Percent'].unique()[:5])
print(df['Cocoa Percent'].value_counts().head())

df['Cocoa Percent - Numerical'] = df['Cocoa Percent'].str[:-1].astype(float)
print(df['Cocoa Percent - Numerical'].describe())

def cococa_fun(cocoa_perc):
    if cocoa_perc >= 90:
        return '> 90%'
    elif cocoa_perc >= 80:
        return '80-89%'
    elif cocoa_perc >= 70:
        return '70-79%'
    elif cocoa_perc >= 60:
        return '60-69%'
    elif cocoa_perc >= 50:
        return '50-59%'
    else:
        return '< 50%'
    
df['Cocoa Percent - Categories'] = df['Cocoa Percent - Numerical'].apply(cococa_fun)

print(df['Cocoa Percent - Categories'].value_counts())

### Ingredients - Number and Type of

In [ ]:
print(df['Ingredients'].value_counts().head())
# In total, 7 different ingredients occur: B, S, S*, C, V, L, Sa

# Split 'Ingredients' column at '-' into No. of and List of Ingredients
# e.g.: '6-B,S,C,V,L,Sa' -> ['6', 'B,S,C,V,L,Sa']
df['Number of Ingredients'] = df['Ingredients'].str.split('-').apply(pd.Series)[0]
df['List of Ingredients'] = df['Ingredients'].str.split('-').apply(pd.Series)[1].str.strip()

# Expand occuring ingredients into 6 columns in order of appearance
# e.g. 'B,S,V' -> ['B', 'S', 'V', None, None, None]
ls_ingredients = df['List of Ingredients'].str.split(',').apply(pd.Series)
for col_nr, col in enumerate(ls_ingredients):
    df[f'Ingredient #{col_nr+1}'] = ls_ingredients[col_nr]

# Create a column for each ingredient indicating whether it's present in the chocolate bar
# e.g. 'B,S,V' -> [True, True, False, False, False, True, False]
unique_ingredients = ['B', 'S', 'S*', 'C', 'V', 'L', 'Sa']

def ingr_fun(cols, ingr):
    if cols.notna().sum() == 0:
        return np.NAN
    else:
        return (cols == ingr).sum() > 0

for i in unique_ingredients:
    #df[f'Ingredient {i}'] = df.filter(like='Ingredient #').apply(lambda x: np.NAN if x.notna().sum() == 0 else (x == i).sum() > 0, axis=1)
    df[f'Ingredient {i}'] = df.filter(like='Ingredient #').apply(ingr_fun, ingr=i, axis=1)

print('\nIngredient Count:\n', df.loc[:,'Ingredient B':'Ingredient Sa'].astype('boolean').sum())


### Most Memorable Characteristics - Number and Type of

In [ ]:
#print(df['Most Memorable Characteristics'].value_counts().head())
# exemplary format: 'spicy, cocoa', 'sandy, sweet, cocoa'

# split characteristics into columns
characts = df['Most Memorable Characteristics'].str.split(',').apply(lambda x: [e.strip() for e in x]).apply(pd.Series)
for col_nr, col in enumerate(characts):
    df[f'Characteristic #{col_nr+1}'] = characts[col_nr]

# create a list with all mentioned characteristics
ls_characts = []
for col in range(len(characts.columns)):
    ls_characts.extend(characts.loc[characts[col].notna()][col].to_list())

all_charact_counts = pd.DataFrame(ls_characts).value_counts().rename_axis('charact').reset_index(name='counts')
#print(all_charact_counts[:20])
top_characts = all_charact_counts[:20]['charact'].tolist()

for i in top_characts:
    df[f'Characteristic - {i}'] = df['Most Memorable Characteristics'].apply(lambda x: i in x)

# Check results
print(all_charact_counts[:20].head())
print(df.filter(like='Characteristic - ').sum().head())

# Counts differ e.g. 'sweet': n=276 in all_charact_counts vs. n=324 in column 'Characteristic - sweet'
print(all_charact_counts[all_charact_counts['charact'].str.contains('sweet')].head())
# Reason: there are entries not solely containing the attribute 'sweet' but slightly more 
#  specific evaluation, e.g. 'very sweet', 'too sweet'
print('Number of unique characteristics containing the word "cocoa":',
      len(all_charact_counts[all_charact_counts['charact'].str.contains('cocoa')]))
# Looking at 'cocoa', there are 51 unique evaluations containing this word, e.g. 'rich cocoa', 
# 'sweet cocoa',... This way of computation should be fine, as e.g. 'sweet cocoa' can certainly 
# be associated with both 'sweet' and 'cocoa', while 'very sweet' is still 'sweet'.

# Data Visualization: Variable Distribution and Rating Means

### Ratings and Sampling Activity

In [ ]:
fig, axs = plt.subplots(figsize=(12, 3), ncols=3)

# Rating Numeric
sns.countplot(data=df, x='Rating', ax=axs[0])
# draw mean (~3.2, which is between x-point 7 (=3.0) and 8 (=3.25))
axs[0].axvline(7.8)

# Rating Categories
rating_cats = ['Unpleasant', 'Disappointing', 'Recommended', 'Highly Recommended', 'Outstanding']
sns.countplot(data=df, x='Rating Categories', ax=axs[1], order=rating_cats)

# Review Date
rd_mean = df.groupby('Review Date')['Rating'].mean().to_frame()
rd_mean['Count'] = df['Review Date'].value_counts()
# Rating Means as second axis
ax2_1 = axs[2].twinx()
sns.countplot(data=df, x='Review Date', ax=axs[2])
sns.stripplot(x=rd_mean.index, y=rd_mean['Rating'], ax=ax2_1)
ax2_1.set_ylim(1,5)
ax2_1.axhline(df['Rating'].mean());

for i in range(len(axs)):
    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=45, horizontalalignment='right')


### Manufacturer and bean origin (top 20)

In [ ]:
fig, axs = plt.subplots(figsize=(18, 9), ncols=2)

vars = ['Company Location', 'Country of Bean Origin']

for nr, var in enumerate(vars):
    # count country occurence and store top 20
    var_count = df[var].value_counts().head(20).to_frame()
    
    # get the aggregated mean for the top 20 countries
    var_means_ls = []
    for i in var_count.index:
        var_means_ls.append(df[df[var] == i]['Rating'].mean())
    var_count['mean'] = var_means_ls

    # Plot
    sns.barplot(x=var_count['count'], y=var_count.index, ax=axs[nr]).set_title(var, fontdict={'size': 15})
    axs2 = axs[nr].twiny()
    sns.stripplot(x=var_count['mean'], y=var_count.index, ax=axs2)
    axs2.set_xlim(1,5)
    axs2.axvline(df['Rating'].mean())
    axs[nr].bar_label(axs[nr].containers[0], labels=var_count['count'])
    axs[nr].set(ylabel=None)


### Cocoa Percent

In [ ]:
fig, axs = plt.subplots(figsize=(18, 6), ncols=2)

# Cocoa Percent - Numerical
cpn_count = df['Cocoa Percent - Numerical'].value_counts().to_frame()
cpn_count['mean'] = df.groupby('Cocoa Percent - Numerical')['Rating'].mean()
cpn_count = cpn_count.sort_index()
cpn_count.index = cpn_count.index.map(str)
# Plot
sns.barplot(x=cpn_count.index, y=cpn_count['count'], ax=axs[0]).set_title('Cocoa Percent - Numerical', fontdict={'size': 15})
axs0_1 = axs[0].twinx()
axs0_1.axhline(df['Rating'].mean(), color='lightgrey')
axs0_1.set_ylim(1,5)
sns.lineplot(x=cpn_count.index, y=cpn_count['mean'], ax=axs0_1)
axs[0].bar_label(axs[0].containers[0], labels=cpn_count['count'])
axs[0].set(xlabel=None)
axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=90);

# Cocoa Content - Categories
cpc_count = df['Cocoa Percent - Categories'].value_counts().to_frame()
cpc_count['mean'] = df.groupby('Cocoa Percent - Categories')['Rating'].mean()
cpc_order = ['< 50%', '50-59%', '60-69%', '70-79%', '80-89%', '> 90%']
cpc_count = cpc_count.sort_index(key=lambda x: x.map(lambda y: cpc_order.index(y)))
# Plot
sns.barplot(x=cpc_count.index, y=cpc_count['count'], order=cpc_order, ax=axs[1]).set_title('Cocoa Percent - Categories', fontdict={'size': 15})
axs1_1 = axs[1].twinx()
axs1_1.axhline(df['Rating'].mean(), color='lightgrey')
axs1_1.set_ylim(1,5)
sns.lineplot(x=cpc_count.index, y=cpc_count['mean'], marker='o', ax=axs1_1)
axs[1].bar_label(axs[1].containers[0], labels=cpc_count['count'])
axs[1].set(xlabel=None);

### Number of Ingredients

In [ ]:
fig, axs = plt.subplots(figsize=(18, 6), ncols=2)

# Number of Ingredients
ni_count = df['Number of Ingredients'].value_counts().to_frame()
ni_count['mean'] = df.groupby('Number of Ingredients')['Rating'].mean()
ni_count = ni_count.sort_index()
#print(ni_count)
# Plot
sns.barplot(x=ni_count.index, y=ni_count['count'], ax=axs[0]).set_title('Number of Ingredients', fontdict={'size': 15})
axs0_1 = axs[0].twinx()
axs0_1.axhline(df['Rating'].mean(), color='lightgrey')
axs0_1.set_ylim(1,5)
sns.lineplot(x=ni_count.index, y=ni_count['mean'], marker='o', ax=axs0_1)
axs[0].bar_label(axs[0].containers[0], labels=ni_count['count'])

# Number of Ingredients - Split by Rating Category
ni_rating = df.groupby(['Number of Ingredients', 'Rating Categories'])['Rating Categories'].value_counts().to_frame().reset_index()
ni_rating['gr_count'] = ni_rating.groupby('Number of Ingredients')['count'].transform('sum')
ni_rating['perc'] = ni_rating['count'] / ni_rating['gr_count']
#print(ni_rating.head())
# Plot
sns.barplot(x=ni_rating['Number of Ingredients'], y=ni_rating['perc'], hue=ni_rating['Rating Categories'], hue_order=rating_cats, ax=axs[1]).set_title('Number of Ingredients - Rating Distribution', fontdict={'size': 15});



### Type of Ingredients

In [ ]:
# Transform data to produce stacked barplots
ingr_ls = []
for ingr in unique_ingredients:
    ingr_cols = ['Ingredient']
    temp_ls = [ingr]
    for cont in [True, False]:
        temp_df = df[df[f'Ingredient {ingr}'] == cont]
        temp_ls.extend([len(temp_df), temp_df['Rating'].mean()])
        ingr_cols.extend([f'{cont} - Count', f'{cont} - Mean'])
        for rc in rating_cats:
            temp_ls.append(len(temp_df[temp_df['Rating Categories'] == rc]) / max(1, len(temp_df)))
            ingr_cols.append(f'{cont} - {rc} - Count')
    ingr_ls.append(temp_ls)

ingrs = pd.DataFrame(ingr_ls, columns=ingr_cols)

ingrs_means_only = (pd.concat([ingrs[['Ingredient', 'True - Mean']].rename(columns={'True - Mean' : 'Mean'}), 
                               ingrs[['Ingredient', 'False - Mean']].rename(columns={'False - Mean' : 'Mean'})],
                               ignore_index=True))
ingrs_means_only['Contained'] = [True for x in range(len(unique_ingredients))] + [False for x in range(len(unique_ingredients))]

ingrs_true_rc_only = ingrs.drop(columns=['True - Count', 'True - Mean', *list(ingrs.filter(regex='False'))])
ingrs_true_rc_only.columns = ['Ingredient', *rating_cats]

ingrs_false_rc_only = ingrs.drop(columns=['False - Count', 'False - Mean', *list(ingrs.filter(regex='True'))])
ingrs_false_rc_only.columns = ['Ingredient', *rating_cats]

# Plot
fig, axs = plt.subplots(figsize=(18, 6), ncols=3)

sns.barplot(x=ingrs['Ingredient'], y=ingrs['True - Count'], ax=axs[0]).set_title('Ingredient Presence', fontdict={'size': 15})
axs[0].bar_label(axs[0].containers[0], labels=ingrs['True - Count'])
axs0_1 = axs[0].twinx()
axs0_1.axhline(df['Rating'].mean(), color='lightgrey')
axs0_1.set_ylim(1,5)
sns.lineplot(x=ingrs_means_only['Ingredient'], y=ingrs_means_only['Mean'], hue=ingrs_means_only['Contained'], hue_order=[True, False], marker='o', palette = sns.color_palette("Set2", 2), ax=axs0_1)

ingrs_true_rc_only.set_index('Ingredient').plot(kind='bar', stacked=True, ax=axs[1]).set_title('Ingredients Present - Rating Distribution', fontdict={'size': 15})
ingrs_false_rc_only.set_index('Ingredient').plot(kind='bar', stacked=True, ax=axs[2]).set_title('Ingredients NOT Present - Rating Distribution', fontdict={'size': 15})

for i in range(len(axs)):
    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=0)


### Most Memorable Characteristics (mentioned or not)

In [ ]:
# Transform data to produce stacked barplots
chrc_ls = []
for chrc in top_characts:
    chrc_cols = ['Characteristic']
    temp_ls = [chrc]
    for cont in [True, False]:
        temp_df = df[df[f'Characteristic - {chrc}'] == cont]
        temp_ls.extend([len(temp_df), temp_df['Rating'].mean()])
        chrc_cols.extend([f'{cont} - Count', f'{cont} - Mean'])
        for rc in rating_cats:
            temp_ls.append(len(temp_df[temp_df['Rating Categories'] == rc]) / max(1, len(temp_df)))
            chrc_cols.append(f'{cont} - {rc} - Count')
    chrc_ls.append(temp_ls)

chrcs = pd.DataFrame(chrc_ls, columns=chrc_cols).sort_values(by='True - Count', ascending=False)

chrcs_means_only = (pd.concat([chrcs[['Characteristic', 'True - Mean']].rename(columns={'True - Mean' : 'Mean'}), 
                               chrcs[['Characteristic', 'False - Mean']].rename(columns={'False - Mean' : 'Mean'})],
                               ignore_index=True))
chrcs_means_only['Mentioned'] = [True for x in range(len(top_characts))] + [False for x in range(len(top_characts))]

chrcs_true_rc_only = chrcs.drop(columns=['True - Count', 'True - Mean', *list(chrcs.filter(regex='False'))])
chrcs_true_rc_only.columns = ['Characteristic', *rating_cats]

# Plot
fig, axs = plt.subplots(figsize=(21, 6), ncols=2)

sns.barplot(x=chrcs['Characteristic'], y=chrcs['True - Count'], ax=axs[0]).set_title('Characteristic Presence', fontdict={'size': 15})
axs[0].bar_label(axs[0].containers[0], labels=chrcs['True - Count'])
axs0_1 = axs[0].twinx()
axs0_1.axhline(df['Rating'].mean(), color='lightgrey')
axs0_1.set_ylim(1,5)
sns.lineplot(x=chrcs_means_only['Characteristic'], y=chrcs_means_only['Mean'], hue=chrcs_means_only['Mentioned'], hue_order=[True, False], palette = sns.color_palette("Set2", 2), marker='o', ax=axs0_1)

chrcs_true_rc_only.set_index('Characteristic').plot(kind='bar', stacked=True, ax=axs[1]).set_title('Characteristic Mentioned - Rating Distribution', fontdict={'size': 15})

for i in range(len(axs)):
    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=45, horizontalalignment='right')